In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
# Load and preprocess images and masks
def preprocess_image_mask(image_path, mask_path, img_size=(128, 128)):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=img_size)
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0

    mask = tf.keras.preprocessing.image.load_img(mask_path, target_size=img_size, color_mode="grayscale")
    mask = tf.keras.preprocessing.image.img_to_array(mask) / 255.0

    return image, mask

def load_dataset(split, image_folder, mask_folder, img_size=(128, 128)):
    images, masks = [], []
    img_dir = os.path.join(image_folder, split)
    mask_dir = os.path.join(mask_folder, split)

    for fname in sorted(os.listdir(img_dir)):
        if fname.endswith('.jpg'):
            img_path = os.path.join(img_dir, fname)
            mask_path = os.path.join(mask_dir, fname.replace('.jpg', '.png'))
            if os.path.exists(mask_path):
                img, mask = preprocess_image_mask(img_path, mask_path, img_size)
                images.append(img)
                masks.append(mask)
            else:
                print(f"Missing mask for {fname}, skipping.")
    return np.array(images), np.array(masks)

In [ ]:
# Paths
image_folder = "/content/gdrive/MyDrive/images"
mask_folder = "/content/gdrive/MyDrive/annotations"

# Load data
X_train, y_train = load_dataset('train', image_folder, mask_folder)
X_val, y_val = load_dataset('val', image_folder, mask_folder)
X_test, y_test = load_dataset('test', image_folder, mask_folder)

print(f"Train: {X_train.shape}, {y_train.shape}")
print(f"Val: {X_val.shape}, {y_val.shape}")
print(f"Test: {X_test.shape}, {y_test.shape}")

In [ ]:
# U-Net model with enhancements
def unet(input_size=(128, 128, 3)):
    inputs = Input(input_size)

    def conv_block(x, filters):
        x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = Conv2D(filters, (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        return x

    c1 = conv_block(inputs, 64)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = conv_block(p1, 128)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = conv_block(p2, 256)

    u1 = UpSampling2D((2, 2))(c3)
    u1 = concatenate([u1, c2])
    c4 = conv_block(u1, 128)

    u2 = UpSampling2D((2, 2))(c4)
    u2 = concatenate([u2, c1])
    c5 = conv_block(u2, 64)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c5)

    return Model(inputs, outputs)

model = unet()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    batch_size=16,
    epochs=20,
    callbacks=callbacks
)

In [ ]:
# Evaluation metrics
def compute_metrics(y_true, y_pred, threshold=0.5):
    y_pred = (y_pred > threshold).astype(np.float32)
    intersection = np.sum(y_true * y_pred)
    union = np.sum(y_true) + np.sum(y_pred) - intersection
    dice = (2 * intersection) / (np.sum(y_true) + np.sum(y_pred) + 1e-6)
    iou = intersection / (union + 1e-6)
    pixel_accuracy = np.sum(y_true == y_pred) / y_true.size
    return iou, dice, pixel_accuracy

def evaluate_metrics_on_dataset(model, X, y, threshold=0.5):
    y_pred = model.predict(X)
    metrics = [compute_metrics(y[i], y_pred[i], threshold) for i in range(len(y))]
    iou, dice, acc = map(np.mean, zip(*metrics))
    print(f"Mean IoU: {iou:.4f}, Dice: {dice:.4f}, Accuracy: {acc:.4f}")
    return iou, dice, acc

evaluate_metrics_on_dataset(model, X_val, y_val)

# Visualization
def visualize_predictions(model, X, y_true, num_samples=5):
    preds = model.predict(X[:num_samples])
    for i in range(num_samples):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(X[i])
        plt.title("Image")
        plt.subplot(1, 3, 2)
        plt.imshow(y_true[i].squeeze(), cmap='gray')
        plt.title("Ground Truth")
        plt.subplot(1, 3, 3)
        plt.imshow(preds[i].squeeze(), cmap='gray')
        plt.title("Prediction")
        plt.show()

visualize_predictions(model, X_val, y_val)
